In [1]:
#### reloads modules automatically - GREAT INCLUSION
using Revise

### !! IMPORTANT!! NEED THIS LINE TO FIND CUSTOM MODULES
!(pwd() in LOAD_PATH) ? push!(LOAD_PATH, pwd()) : nothing

using DataFrames
using Distributed
using FastGraphs
using Graphs
using JuMP
using LinearAlgebra
using SharedArrays
using SparseArrays
using SupportFunctions
using HiGHS
using GraphWrappers



using StatsBase

# install kernal with # of threads:
# https://github.com/JuliaLang/IJulia.jl/issues/882
#
# using IJulia
# IJulia.installkernel("Julia 1.8 20 Threads", env=Dict("JULIA_NUM_THREADS" => "20",))

[ Info: Precompiling FastGraphs [top-level]


##  Read in edgeweights and convert to adjacency, graph, etc.

In [3]:
fast_graph.dims

(2458, 7142)

In [2]:


fp_sa = joinpath(pwd(), "shared_array_tmp")
fp_egl = "/Users/jsyme/Documents/Projects/FY23/KPP_RIR/edge_lists/dprk_iran_edges.egl"

# RUN TO AVOID shm_open() error if it shows up
#  see: https://github.com/JuliaLang/julia/issues/15415
#=
GC.gc() 
=# 
fast_graph = read_egl(fp_egl);



# Try Krebs (2002) Terrorist Network for validation
- Source: https://networks.skewed.de/net/terrorists_911

In [3]:
fp_krebs = "/Users/jsyme/Documents/Projects/FY23/KPP_RIR/krebs_network.csv/edges.csv"
fp_krebs_out = "/Users/jsyme/Documents/Projects/FY23/KPP_RIR/edge_lists/krebs_test.egl"

mat_edges = split.(readlines(fp_krebs), ",");
mat_edges = mat_edges[2:end]
edges_out = sort(sort.(collect.(Set([sort(x) for x in mat_edges]))))
edges_out = [edges_out; [reverse(x) for x in edges_out]]
edges_out = join.(edges_out, " ")

edges_out = join(edges_out, "\n")
write(fp_krebs_out, edges_out)


fast_graph_krebs = read_egl(fp_krebs_out);


In [24]:
?stochastic_gradient_descent

search: stochastic_gradient_descent



# Constructs

stochastic*gradient*descent(     n*nodes::Int64,     graph*fast::GraphWrapper;     epsilon::Float64 = 10^(-6),     max*iter::Int64 = 100,     S::Union{Vector{Int64}, UnitRange{Int64}, Nothing} = nothing,     randomize*swap_count::Bool = false )

## Function Arguments

  * `n_nodes`: number of nodes to remove
  * `graph_fast`: FastGraph storing information for running swaps

## Keyword Arguments

  * `distance_algorithm`: distance algorithm to use to update distance matrix. Passed to get*distance*matrices(). Default is :auto
  * `epsilon`: convergence threshold
  * `log_info`: print outputs?
  * `max_iter`: maximum number of iterations to allow
  * `max_iter_no_improvement`: maximum number of iterations without improving the objective function to allow
  * `randomize_swap_count`: if true, randomizes number of swaps on each   iteration. Otherwise, swaps one at a time
  * `S`: optional set of indices to start with. Must have length `n_nodes`
  * `...kwargs`: passed to get*default*kpp_nodes()


In [8]:




@time F_krebs, S_krebs, graph_krebs_frag = stochastic_gradient_descent(
    3,
    fast_graph_krebs; 
    centrality = :eigenvector,
    epsilon = 10^(-8),
    log_info = false,
    max_iter = 5000,
    max_iter_no_improvement = 1000,
    randomize_swap_count = true,
    #S = [3, 22, 8]#[4, 25, 26],
)



F_krebs, S_krebs


  7.100349 seconds (21.47 M allocations: 1.212 GiB, 5.09% gc time, 93.86% compilation time)


(0.7970791238763186, [4, 25, 26])

In [15]:
dict_test = Dict("a" => 5, "b" => 19, "c" => -2, "t" => 20)

Dict{String, Int64} with 4 entries:
  "c" => -2
  "t" => 20
  "b" => 19
  "a" => 5

In [27]:
@time F_krebs, S_krebs, graph_krebs_frag = greedy_optimization(
    3,
    fast_graph_krebs; 
    centrality = :eigenvector,
    epsilon = 10^(-8),
    log_info = false,
    max_iter = 5000,
    max_iter_no_improvement = 1000,
    randomize_swap_count = true,
    #S = [3, 22, 8]#[4, 25, 26],
)


  0.083185 seconds (354.38 k allocations: 59.021 MiB, 46.95% gc time)


(0.7970791238763186, [4, 25, 26], SimpleGraph{Int64}(106, [[2], [1, 3, 4, 13, 24, 25, 26, 35, 46, 57], [2, 7, 26], [2, 5, 6, 25, 46, 57], [4, 6, 8, 9, 10], [4, 5, 18], [3, 8], [5, 7, 10, 16, 17, 18], [5, 10, 11], [5, 8, 9, 11, 12, 14, 15, 16]  …  [38, 40, 44, 45, 47, 51], [40, 44, 50], [39, 40, 43, 53], [39, 40, 43, 52, 56, 58], [39, 49], [39, 49], [40, 53, 58], [2, 4], [40, 53, 56], [40]]))

In [29]:
@time F, S, graph_out = greedy_optimization(
    3,
    fast_graph; 
    centrality = :eigenvector,
    epsilon = 10^(-8),
    log_info = true,
    max_iter = 5000,
    max_iter_no_improvement = 1000,
    randomize_swap_count = true,
    #S = [3, 22, 8]#[4, 25, 26],
)


LoadError: InterruptException:

In [50]:
@time path = Graphs.dijkstra_shortest_paths(fast_graph.graph, 1)
propertynames(path)

path.pre



  0.000737 seconds (38 allocations: 206.125 KiB)


(:parents, :dists, :predecessors, :pathcounts, :closest_vertices)

In [34]:
2458*2457*2456/6

2.472089256e9

In [18]:
@time F, S, graph_out = stochastic_gradient_descent(
    3,
    fast_graph; 
    epsilon = 0.0,#(10^(-10)),
    log_info = true,
    max_iter = 1000,
    max_iter_no_improvement = 100,
    randomize_swap_count = true,
    #S = [3, 22, 8]#[4, 25, 26],
)

┌ Info: 
│ 
└ ** keeping swap! 0 - F = 0.8262404466978552, F_0 = 0, F_try = 0.8658997962469739
┌ Info: 
│ 
└ ** keeping swap! 2 - F = 0.8658997962469739, F_0 = 0.8262404466978552, F_try = 0.8659258536942422
[ Info: 10 iterations complete with value 0.8659258536942422 (previous 0.8658997962469739)
┌ Info: 
│ 
└ ** keeping swap! 17 - F = 0.8659258536942422, F_0 = 0.8658997962469739, F_try = 0.8660849033755482
┌ Info: 
│ 
└ ** keeping swap! 18 - F = 0.8660849033755482, F_0 = 0.8659258536942422, F_try = 0.8660983205296491
[ Info: 20 iterations complete with value 0.8660983205296491 (previous 0.8660849033755482)
[ Info: 30 iterations complete with value 0.8660983205296491 (previous 0.8660849033755482)
[ Info: 40 iterations complete with value 0.8660983205296491 (previous 0.8660849033755482)
┌ Info: 
│ 
└ ** keeping swap! 44 - F = 0.8660983205296491, F_0 = 0.8660849033755482, F_try = 0.8661094245213777
┌ Info: 
│ 
└ ** keeping swap! 49 - F = 0.8661094245213777, F_0 = 0.8660983205296491, F_tr

 95.192659 seconds (35.95 M allocations: 93.239 GiB, 18.75% gc time)


[ Info: Converged in 395 iterations.


(0.8717148154218237, [82, 889, 1060], SimpleGraph{Int64}(3258, [[1051, 2214], [379, 909, 921, 1632, 1831, 1974, 2204], [7, 486, 1224, 1226, 1396, 1596, 1600, 1831, 2199], [34, 1948], [525], [587], [3], [619, 1396], [484, 1931], [11]  …  [207], [349, 1785], [908], [56], [1632], [586, 592, 1172, 1251, 1667], [308], [308], [400, 486], [593]]))

In [165]:
@time d, d_inv = get_distance_matrices(
    fast_graph.A; 
    algorithm = :bellman_ford,
    graph = fast_graph.graph
);

#fast_graph.dims[1]

  0.380267 seconds (55.83 k allocations: 322.889 MiB, 15.50% gc time)


In [17]:
using Graphs.Parallel


In [36]:

#@time d_tmp = Parallel.dijkstra_shortest_paths(fast_graph.graph);
@time d_tmp = Parallel.bellman_ford_shortest_paths(fast_graph.graph);

LoadError: MethodError: no method matching bellman_ford_shortest_paths(::SimpleGraph{Int64})
[0mClosest candidates are:
[0m  bellman_ford_shortest_paths(::AbstractGraph{U}, [91m::AbstractVector{<:Integer}[39m) where U<:Integer at ~/.julia/packages/Graphs/7SMZs/src/Parallel/shortestpaths/bellman-ford.jl:1
[0m  bellman_ford_shortest_paths(::AbstractGraph{U}, [91m::AbstractVector{<:Integer}[39m, [91m::AbstractMatrix{T}[39m) where {U<:Integer, T<:Real} at ~/.julia/packages/Graphs/7SMZs/src/Parallel/shortestpaths/bellman-ford.jl:1

In [149]:
g = fast_graph_krebs.graph
tk = nothing
@time for i in 1:fast_graph_krebs.dims[1]
    tk = Graphs.dijkstra_shortest_paths(g, i)
end

  0.001112 seconds (1.35 k allocations: 383.297 KiB)


In [150]:
g = fast_graph_krebs.graph
tkb = nothing
@time for i in 1:fast_graph_krebs.dims[1]
    tkb = Graphs.bellman_ford_shortest_paths(g, i)
end

  0.000176 seconds (895 allocations: 138.531 KiB)


In [140]:
@time Parallel.dijkstra_shortest_paths(fast_graph.graph)

  1.481426 seconds (87.32 k allocations: 509.222 MiB, 5.70% gc time)


Graphs.Parallel.MultipleDijkstraState{Int64, Int64}([0 5 … 8 9223372036854775807; 5 0 … 6 9223372036854775807; … ; 8 6 … 0 9223372036854775807; 9223372036854775807 9223372036854775807 … 9223372036854775807 0], [0 1974 … 1558 0; 1051 0 … 1558 0; … ; 1051 379 … 0 0; 0 0 … 0 0])

In [29]:
check = 1:n_v
println("parallel:")
@time tmp_par = Graphs.Parallel.dijkstra_shortest_paths(g, check)

println("loop:")
@time for i in check
    tmp = Graphs.dijkstra_shortest_paths(g, i)
end

LoadError: UndefVarError: n_v not defined

In [52]:
@time adj, arr_store_cols, arr_store_rows, S, F = stochastic_gradient_descent(
    3,
    fast_graph; 
    epsilon = 0.0,#(10^(-10)),
    max_iter = 5000,
    max_iter_no_improvement = 1000,
    print_info = true,
    randomize_swap_count = true,
    #S = [3, 22, 8]#[4, 25, 26],
)




** keeping swap! 0 - F = 0.8262404466978552, F_0 = 0, F_try = 0.8266472622279978


** keeping swap! 4 - F = 0.8266472622279978, F_0 = 0.8262404466978552, F_try = 0.8266997323692662


** keeping swap! 5 - F = 0.8266997323692662, F_0 = 0.8266472622279978, F_try = 0.8267871867441089


** keeping swap! 6 - F = 0.8267871867441089, F_0 = 0.8266997323692662, F_try = 0.8328234673952377


** keeping swap! 10 - F = 0.8328234673952377, F_0 = 0.8267871867441089, F_try = 0.8328348169426764


** keeping swap! 20 - F = 0.8328348169426764, F_0 = 0.8328234673952377, F_try = 0.832897118892513


** keeping swap! 21 - F = 0.832897118892513, F_0 = 0.8328348169426764, F_try = 0.8329520801615857


** keeping swap! 36 - F = 0.8329520801615857, F_0 = 0.832897118892513, F_try = 0.8338149632986407


** keeping swap! 54 - F = 0.8338149632986407, F_0 = 0.8329520801615857, F_try = 0.8339369756544097


LoadError: InterruptException:

In [47]:
propertynames(fast_graph_krebs)
fast_graph_krebs.vertex_names[[4, 25, 26]]

3-element Vector{String}:
 "11"
 "30"
 "31"

In [11]:
fragmentation(
    fast_graph_krebs.A;
    D_invs = fast_graph_krebs.D_inv,
    graph = fast_graph_krebs.graph,
)

0.5989335448616254

In [13]:
mat = copy(fast_graph_krebs.D_inv)

for i in [4, 25, 26]
    mat[:, i] .= 0
    mat[i, :] .= 0
end

#dropzeros!(mat);

fragmentation(
    fast_graph_krebs.A;
    D_invs = mat,#fast_graph_krebs.D_inv,
    graph = fast_graph_krebs.graph,
)

0.6533139432399084

# Check Borgatti Examples 5a and 5b for validation

In [70]:
examples = FastGraphs.Examples()
mat_5a = sparse(examples.get_example(:borgatti_figure_5a)); # reported vaue is 
mat_5b = examples.get_example(:borgatti_figure_5b); #
fragmentation(mat_5a)




0.5555555555555556

In [5]:
#fast_graph.vertex_names

In [16]:
fast_graph_krebs

test1_FastGraph(sparse([2, 1, 3, 4, 13, 24, 35, 46, 57, 60  …  3, 46, 2, 62, 2, 5, 6, 46, 57, 61], [1, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  60, 60, 61, 61, 62, 62, 62, 62, 62, 62], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 62, 62), false, ["0", "1", "10", "11", "12", "13", "14", "15", "16", "17"  …  "56", "57", "58", "59", "6", "60", "61", "7", "8", "9"], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0 1.0 … 2.0 2.0; 1.0 0.0 … 1.0 1.0; … ; 2.0 1.0 … 0.0 1.0; 2.0 1.0 … 1.0 0.0], [0.0 1.0 … 0.5 0.5; 1.0 0.0 … 1.0 1.0; … ; 0.5 1.0 … 0.0 1.0; 0.5 1.0 … 1.0 0.0], (62, 304), SimpleGraph{Int64}(152, [[2], [1, 3, 4, 13, 24, 35, 46, 57, 60, 61, 62], [2, 7, 60], [2, 6, 8, 18, 20, 21, 22, 23, 25, 26  …  29, 30, 32, 33, 34, 36, 37, 38, 39, 40], [6, 8, 9, 10, 62], [4, 5, 18, 62], [3, 8], [4, 5, 7, 10, 16, 17, 18], [5, 10, 11], [5, 8, 9, 11, 12, 14, 15, 16]  …  [39, 40, 43, 52, 56, 5

In [18]:
@time fragmentation(
    fast_graph_krebs.A;
    D_invs = fast_graph_krebs.D_inv,
    graph = fast_graph_krebs.graph,
)


  0.000020 seconds (6 allocations: 320 bytes)


0.5989335448616254

In [31]:
F

0.6547241318526356

In [28]:
fast_graph_krebs.vertex_names[[4, 25, 26]]

3-element Vector{String}:
 "11"
 "30"
 "31"

In [49]:
using CSV

i, j, v = findnz(fast_graph.A)

df_out = DataFrame(
    :i => i,
    :j => j
)

CSV.write("/Users/jsyme/Documents/Projects/FY23/KPP_RIR/temp_vals.csv", df_out)

"/Users/jsyme/Documents/Projects/FY23/KPP_RIR/temp_vals.csv"

In [15]:
examples = FastGraphs.Examples()
mat_5a = examples.get_example(:borgatti_figure_5a);
mat_5b = examples.get_example(:borgatti_figure_5b);

fragmentation(mat_5b)

0.7148148148148149

In [13]:
#fragmentation(mat_5b; use_distance_weighting = false)
using Distributed
@everywhere GC.gc()

In [12]:
fp_egl_out = "/Users/jsyme/Documents/Projects/FY23/KPP_RIR/edge_lists/dprk_iran_edges_undirected.egl"
fast_graph_un = TEST_read_egl(fp_egl_out);

In [14]:
#fast_graph_un.D

In [26]:
# create an undirected version

#fp_egl_out = "/Users/jsyme/Documents/Projects/FY23/KPP_RIR/edge_lists/dprk_iran_edges_undirected.egl"

mat_edges = split.(readlines(fp_egl), " ");
edges_out = sort(sort.(collect.(Set([sort(x) for x in mat_edges]))))
edges_out = join.(edges_out, " ")
#edges_out = join(edges_out, "\n")

#write(fp_egl_out, edges_out)



3571-element Vector{String}:
 "10th_Olefin IRAN"
 "10th_Olefin Technip"
 "2ANS Cabinet"
 "2ANS HK_Express"
 "2ANS Habjanggang_Trading"
 "2ANS Nam_Dae_Cheon"
 "2ANS Pusong"
 "2ANS SSRC"
 "2ANS Tangun"
 "2_Econ 5MIB"
 "2_Econ DPRK"
 "2_Econ KOMID"
 "2_Econ KOMID_Iran"
 ⋮
 "Uganda_PF_Kabalye Uganda_Police"
 "Uganda_PF_Masindi Uganda_Police"
 "Uganda_PF_Wantembo Uganda_Police"
 "Unaforte Unaforte_Italy"
 "Unaforte Unaforte_Yanbian"
 "Vanguard_Shipping_Safety Virtue_Base"
 "Virtue_Base Yuan_Tai"
 "Voetsch Voetsch_China"
 "Volgen Yevdokimov"
 "Win_More Yingjen_Fishery"
 "Winning_International Xingyuxin"
 "Wooheng_Shipping Wynn_East"

In [21]:
#fast_graph_un = TEST_read_egl(fp_egl_out);
lns = readlines(fp_egl_out);

In [32]:
#length(edges_out)


In [ ]:
#### fragmentation(
    fast_graph.A;
    fast_graph.D_inv,
    
    

In [4]:
"""


Return indices of vertices ordered by specified centrality metric. Always
    ordered from smallest to largest.

##  Constructs

```
get_vertices_by_centrality(
    n::Int64,
    measure::Symbol;
    ordering::Symbol = :top
)
```


##  Function Arguments

- `n`: number of vertices to return
- `measure`: any of the following symbols:
    * :betweenness
    * :degree
    * :eigenvector


##  Keyword Arguments

- `ordering`: :top or :bottom. Use :top to get retrieve the top n vertices
    by the specified centrality or :bottom to get the bottom n. 
"""
function get_vertices_by_centrality(
    graph_fast::test1_FastGraph,
    n::Int64,
    measure::Symbol;
    ordering::Symbol = :top
)
    
    vec = graph_fast.centrality(measure)
    
    if measure == :betweenness
        
    else
        """
        PLACEHOLDER FOR OTHER MEASURES
        """
    end
    
end

2458-element Vector{Float64}:
 0.0002396512792609862
 0.027002154416108398
 0.023321535730591172
 0.0
 0.0
 0.0
 0.0
 0.0007659429483533721
 0.0
 0.0
 0.0
 0.0002396512792609862
 0.00022733915818985543
 ⋮
 0.00018001090025831094
 0.0
 0.0
 0.0
 0.0033864753509096055
 0.0
 0.0
 0.0
 0.0
 0.0
 0.005659519743227193
 0.0

In [27]:
df = get_alignment_data_frame(fast_graph) 
using CSV
CSV.write("/Users/jsyme/Desktop/graph_check.csv", df)

"/Users/jsyme/Desktop/graph_check.csv"

In [23]:
function get_alignment_data_frame(
    fast_graph::test1_FastGraph;
    field_column::Symbol = :column,
    field_row::Symbol = :row,
    field_weight::Symbol = :weight
)::Union{DataFrame, Nothing}
    
    i, j, v = findnz(fast_graph.A)
    
    df_out = DataFrame(
        Dict(
            field_row => i,
            field_column => j,
            field_weight => fast_graph.w
        )
    )
    
    # sort
    sort!(df_out, [field_row, field_column])
    
    return df_out
end


get_alignment_data_frame (generic function with 1 method)

In [11]:
fragmentation(
    fast_graph.A; 
    D_invs = fast_graph.D_inv,
    graph = fast_graph.graph,
    use_distance_weighting = false
)

0.9988174137889354

##  Precompile distance matrices to speed up fragmentation calculations

In [31]:
D = SharedArray{Float64}(size(adj));
@time D, D_inv = get_distance_matrices(adj, D);

#
# NOTE ON IMPROVEMENTS:
#  (1) split into connected components first
#  (2) breadth-first search for parents (I need to understand what this actually is)
#  (3) parameterization of search
#

  3.566762 seconds (2.42 M allocations: 768.913 MiB, 4.21% gc time, 24.77% compilation time)


##  Calculate fragmentation

In [36]:
@time fract = fragmentation(
    adj; 
    D_invs = D_inv,
    graph = graph,
    use_distance_weighting = true
)


  0.330568 seconds (1.70 M allocations: 81.814 MiB, 98.63% compilation time)


0.8262404466978552

##  Build "stochastic gradient descent"
- see process at https://www.rdocumentation.org/packages/influenceR/versions/0.1.0/topics/keyplayer
- Add in link-based approach
    - with Pyntacle
    - fragmentation, look at cut set and minimum cut set

In [306]:
@time adj, arr_store_cols, arr_store_rows, S = stochastic_gradient_descent(
    1,
    fast_graph; 
    epsilon = (10^(-10)),
    max_iter = 5000,
    randomize_swap_count = true,
    #S = S,
);



** keeping swap! 2 - F = 0.8263980457479595, F_0 = 0, F_try = 0.8264421164324398


** keeping swap! 41 - F = 0.8264421164324398, F_0 = 0.8263980457479595, F_try = 0.8264735232883395
100 iterations complete with value 0.8264735232883395 (previous 0.8264421164324398)
200 iterations complete with value 0.8264735232883395 (previous 0.8264421164324398)
300 iterations complete with value 0.8264735232883395 (previous 0.8264421164324398)


** keeping swap! 369 - F = 0.8264735232883395, F_0 = 0.8264421164324398, F_try = 0.8264775032640993
400 iterations complete with value 0.8264775032640993 (previous 0.8264735232883395)
500 iterations complete with value 0.8264775032640993 (previous 0.8264735232883395)


** keeping swap! 587 - F = 0.8264775032640993, F_0 = 0.8264735232883395, F_try = 0.8264782747015142
600 iterations complete with value 0.8264782747015142 (previous 0.8264775032640993)
700 iterations complete with value 0.8264782747015142 (previous 0.8264775032640993)
800 iterations complete 

In [309]:
v_centrality = betweenness_centrality(fast_graph.graph)

2458-element Vector{Float64}:
 0.0002396512792609862
 0.027002154416108398
 0.023321535730591172
 0.0
 0.0
 0.0
 0.0
 0.0007659429483533721
 0.0
 0.0
 0.0
 0.0002396512792609862
 0.00022733915818985543
 ⋮
 0.00018001090025831094
 0.0
 0.0
 0.0
 0.0033864753509096055
 0.0
 0.0
 0.0
 0.0
 0.0
 0.005659519743227193
 0.0

In [302]:
m = size(adj2)[1]
n_nodes = length(S)
arr_store_cols = zeros(n_nodes, m)
arr_store_rows = zeros(n_nodes, m)
arr_store_cols_transient = zeros(n_nodes, m)
arr_store_rows_transient = zeros(n_nodes, m);

In [303]:
@time swap!(
    adj2,
    S, S, S,
    arr_store_cols,
    arr_store_rows,
    arr_store_cols_transient,
    arr_store_rows_transient,
)

  0.003249 seconds (119 allocations: 1.131 MiB)


In [111]:
#adj[:, 90]
#maximum(arr_store_cols[1, :])
for i in enumerate(S)
    
    ind, i = i
    
    m0 = maximum(fast_graph.D_inv[i, :])
    m1 = maximum(fast_graph.D_inv[:, i])
    
    m_total = maximum([m0, m1])
    
    #(m_total > 0) ? println(i) : nothing
    
    println(maximum(arr_store_rows[ind, :]))
end

0.0
0.5
0.2
0.25
0.3333333333333333
0.5
0.25
0.25
0.25
0.3333333333333333
0.5
0.25
0.3333333333333333
0.25
0.3333333333333333
0.5
0.25
0.5
0.5
0.5
0.25
0.3333333333333333
0.5
0.3333333333333333
0.25
0.25
0.5
0.14285714285714285
0.5
0.3333333333333333
0.5
0.5
0.3333333333333333
0.5
0.5
0.3333333333333333
0.5
0.3333333333333333
0.3333333333333333
0.5
0.3333333333333333
0.5
0.5
0.3333333333333333
1.0
0.5
0.2
0.5
0.16666666666666666
1.0842021724855044e-19
0.5
0.5
0.5
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.5
0.5
0.3333333333333333
0.25
0.3333333333333333
0.5
0.3333333333333333
1.0842021724855044e-19
0.25
0.5
0.5
0.3333333333333333
0.5
0.3333333333333333
0.5
0.5
0.5
0.3333333333333333
0.3333333333333333
1.0
0.3333333333333333
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.16666666666666666
0.25
0.5
0.5
0.3333333333333333
0.3333333333333333
0.5
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.5
0.3333333333333333
0.5


5×5 Matrix{Float64}:
 0.47956    0.622032  0.141792  0.641025  0.212467
 0.28124    0.824644  0.399715  0.219051  0.228287
 0.870848   0.808636  0.348541  0.649718  0.833938
 0.412432   0.295449  0.29573   0.18228   0.893942
 0.0946407  0.716407  0.499005  0.219982  0.955113

In [261]:
adj = rand(5, 5)
m = size(adj)[1]
n_nodes = 2
S = Vector{Int64}(sample(1:m, n_nodes; replace = false))
arr_store_cols = zeros(n_nodes, m)
arr_store_rows = zeros(n_nodes, m)
arr_store_cols_transient = zeros(n_nodes, m)
arr_store_rows_transient = zeros(n_nodes, m);
swap!(
    adj,
    S, S, S,
    arr_store_cols,
    arr_store_rows,
    arr_store_cols_transient,
    arr_store_rows_transient,
)

In [262]:
adj

5×5 Matrix{Float64}:
 0.750806  0.0  0.335105  0.904969   0.0
 0.0       0.0  0.0       0.0        0.0
 0.714849  0.0  0.622389  0.246009   0.0
 0.215334  0.0  0.451835  0.0709246  0.0
 0.0       0.0  0.0       0.0        0.0

In [263]:
arr_store_cols

2×5 Matrix{Float64}:
 0.464767  0.622727  0.655561  0.765506  0.0906651
 0.241644  0.316475  0.11871   0.59103   0.266664

In [264]:
arr_store_rows

2×5 Matrix{Float64}:
 0.514229  0.622727   0.476625  0.852058  0.316475
 0.107181  0.0906651  0.619353  0.232175  0.266664

In [265]:
swap!(
    adj,
    S, [1], [2],
    arr_store_cols,
    arr_store_rows,
    arr_store_cols_transient,
    arr_store_rows_transient,
)

In [266]:
adj

5×5 Matrix{Float64}:
 0.750806  0.464767   0.335105  0.904969   0.0
 0.514229  0.622727   0.476625  0.852058   0.316475
 0.714849  0.655561   0.622389  0.246009   0.0
 0.215334  0.765506   0.451835  0.0709246  0.0
 0.0       0.0906651  0.0       0.0        0.0

In [267]:
arr_store_cols

2×5 Matrix{Float64}:
 0.0       0.0       0.0      0.0      0.0
 0.241644  0.316475  0.11871  0.59103  0.266664

In [268]:
arr_store_rows

2×5 Matrix{Float64}:
 0.0       0.0        0.0       0.0       0.0
 0.107181  0.0906651  0.619353  0.232175  0.266664

In [105]:
for i in enumerate(S)
    
    ind, i = i
    
    m0 = maximum(adj[i, :])
    m1 = maximum(adj[:, i])
    
    m_total = maximum([m0, m1])
    
    #(m_total > 0) ? println(i) : nothing
    
    println(maximum(arr_store_rows[ind, :]))
end

0.0
0.5
0.2
0.25
0.3333333333333333
0.5
0.25
0.25
0.25
0.3333333333333333
0.5
0.25
0.3333333333333333
0.25
0.3333333333333333
0.5
0.25
0.5
0.5
0.5
0.25
0.3333333333333333
0.5
0.3333333333333333
0.25
0.25
0.5
0.14285714285714285
0.5
0.3333333333333333
0.5
0.5
0.3333333333333333
0.5
0.5
0.3333333333333333
0.5
0.3333333333333333
0.3333333333333333
0.5
0.3333333333333333
0.5
0.5
0.3333333333333333
1.0
0.5
0.2
0.5
0.16666666666666666
1.0842021724855044e-19
0.5
0.5
0.5
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.5
0.5
0.3333333333333333
0.25
0.3333333333333333
0.5
0.3333333333333333
1.0842021724855044e-19
0.25
0.5
0.5
0.3333333333333333
0.5
0.3333333333333333
0.5
0.5
0.5
0.3333333333333333
0.3333333333333333
1.0
0.3333333333333333
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.16666666666666666
0.25
0.5
0.5
0.3333333333333333
0.3333333333333333
0.5
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.5
0.3333333333333333
0.5


In [95]:
S

100-element Vector{Int64}:
   90
   95
  105
  121
  157
  192
  199
  239
  277
  293
  317
  377
  407
    ⋮
 2114
 2133
 2197
 2294
 2302
 2305
 2312
 2359
 2394
 2432
 2436
 2456

In [260]:
fast_graph.A[2049, :]

2458-element SparseVector{Float64, Int64} with 6 stored entries:
  [34  ]  =  1.0
  [343 ]  =  1.0
  [484 ]  =  1.0
  [1601]  =  1.0
  [1935]  =  1.0
  [1948]  =  1.0

##  After finishing SGD
- look to network-fragmentation-optimization-error.pdf (in project folder--from Colin)
- look at implementing

In [54]:
#using Random
rng = Int64
#Random.rand(rng, Tuple(1:50))


Int64

In [55]:
rng = Int64
simp = Random.SamplerSimple(rng, 1:50)



Random.SamplerSimple{DataType, UnitRange{Int64}, Any}(Int64, 1:50)

In [58]:
rng = MersenneTwister()
sp = Random.Sampler(rng, 1:20) # or Random.Sampler(MersenneTwister, 1:20)


16

In [80]:
sample(1:50, 5; replace = false)

5-element Vector{Int64}:
 38
 13
 47
 22
 20

In [65]:
rng = Int64
sp = Random.SamplerSimple(rng, 1:20) # or Random.Sampler(MersenneTwister, 1:20)


Random.SamplerSimple{DataType, UnitRange{Int64}, Any}(Int64, 1:20)

In [68]:
rand(rng, sp)

LoadError: MethodError: no method matching rand(::Type{Int64}, ::Random.SamplerSimple{DataType, UnitRange{Int64}, Any})
[0mClosest candidates are:
[0m  rand(::Type{X}) where X at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Random/src/Random.jl:260
[0m  rand(::Type{X}, [91m::Tuple{Vararg{Int64, N}} where N[39m) where X at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Random/src/Random.jl:290
[0m  rand(::Type{X}, [91m::Integer[39m, [91m::Integer...[39m) where X at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Random/src/Random.jl:293
[0m  ...

# Next steps for May 1
- Implement Gradient descent
- Think about bayesian approach (would be very straightforwrd to implement bayesian updating on binomial assumption (Bernoulli) to updating edges
- Potential integration of Multigraphs (layered)--(see community detection from https://people.maths.ox.ac.uk/porterm/papers/multislice.pdf)

In [178]:

"""
y <= x1
y <= x2
y >= x1 - (1 - b)M
y >= x2 - bM


0, 0 -> y = 0
1, 0 -> b = 0, y = 0
0, 1 -> b = 1, y = 0
1, 1 -> y = 1

"""
function TEST_maximize_fragmentation_jump(
    D_inv::Matrix{Float64};
    n_nodes_remove::Int64 = 1
)
    M = 10
    
    # initialize
    n_nodes_remove = maximum([n_nodes_remove, 1])
    n, n = size(D_inv)
    n_pairs = n^2
    
    model = Model(optimizer_with_attributes(HiGHS.Optimizer))
    
    sums_row = sum.(eachrow(D_inv))
    sums_col = sum.(eachrow(transpose(D_inv)))
    
    # add decision variable (number of vertices less 1)
    @variable(model, 0 <= x[1:n] <= 1, Bin)
    @variable(model, 0 <= y[1:n, 1:n] <= 1, Bin)
    @variable(model, 0 <= b[1:n, 1:n] <= 1, Bin)
    
    @constraint(model, sum(x[i] for i in 1:n) == n - n_nodes_remove)
    
    # bounds on intersections of removed rows
    @constraint(model, c1[i in 1:n, j in 1:n], y[i, j] <= x[j])
    @constraint(model, c2[i in 1:n, j in 1:n], y[i, j] <= x[i])
    @constraint(model, c3[i in 1:n, j in 1:n], x[i] - (1 - b[i, j])*M <= y[i, j])
    @constraint(model, c4[i in 1:n, j in 1:n], x[j] - b[i, j]*M <= y[i, j])

    
    # NOT EFFICIENT, THERE IS A BETTER WAY
    @objective(
        model, 
        Max, 
        sum(
            x[i]*sums_row[i] + x[i]*sums_col[i] for i in 1:n
        ) - sum(
            y[i, j]*D_inv[i, j] for i in 1:n, j in 1:n
        )
    );
    
    optimize!(model)
    
    return model
end



@time w_mod_jump = TEST_maximize_fragmentation_jump(
    D_test_inv;
    n_nodes_remove = 2
);


Running HiGHS 1.5.1 [date: 1970-01-01, git hash: 93f1876e4]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1000001 rows, 500500 cols, 2500500 nonzeros
999501 rows, 500500 cols, 2499500 nonzeros

Solving MIP model with:
   999501 rows
   500500 cols (500500 binary, 0 integer, 0 implied int., 0 continuous)
   2499500 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   3248.136364     -inf                 inf        0      0      0         0     7.5s


LoadError: InterruptException:

SparseMatrixCSC

In [174]:
# try setting up model for fragmentation optimization



"""
Generate the identify matrix without the nth element.
"""
function I_n(
    n_row::Int64,
    n::Int64
) 
    vec = Int64.(ones(n_row))
    vec[n] = 0
    
    return diagm(vec)
end



function TEST_maximize_fragmentation_linear(
    adj::SparseMatrixCSC,
    D_inv::Matrix{Float64};
    vertex_indices::Union{Vector{Int64}, Nothing} = nothing
)
    
    n, m = size(adj)
    vertex_indices = isa(vertex_indices, Nothing) ? collect(1:size(adj)[1]) : [x for x in 1:n if x in vertex_indices]
    vec_fracs = -1.0*ones(length(vertex_indices))
    
    D_inv_cur = copy(D_inv)
    
    col_cur = ones(n)
    row_cur = ones(n)
    
    for i in vertex_indices

        # preserve for replacement again
        for j in 1:n
            @inbounds col_cur[j] = D_inv_cur[j, i]
            @inbounds row_cur[j] = D_inv_cur[i, j]
        end
        
        # set to 0
        D_inv_cur[:, i] .= 0
        D_inv_cur[i, :] .= 0
        
        fract = fragmentation(
            adj; 
            D_invs = D_inv_cur,
            use_distance_weighting = true
        )
        
        # reset
        D_inv_cur[:, i] = col_cur
        D_inv_cur[i, :] = row_cur
        
        vec_fracs[i] = fract

    end
    
    print(minimum(vec_fracs))
    
    w = findall(x -> ((x == maximum(vec_fracs)) & (x > 0)), vec_fracs)
    
    return w

end


@time w_mod = TEST_maximize_fragmentation_linear(
    adj, 
    D_lower_invs
);

0.826240777861734  8.227998 seconds (31.77 M allocations: 547.776 MiB, 0.94% gc time, 2.39% compilation time)


In [134]:
using Statistics

In [62]:
vec = Graphs.Parallel.dijkstra_shortest_paths(graph, 2).dists
sum(1 ./ vec[vec .> 0])

539.9940836940837

  2.769581 seconds (800.46 k allocations: 685.232 MiB, 10.90% gc time)
  0.108650 seconds (618.16 k allocations: 29.473 MiB, 96.59% compilation time)


0.8262404466978552

In [25]:
@time fragmentation(
    adj; 
    D_invs = D_lower_invs,
    use_distance_weighting = true
)

  0.003712 seconds (5.06 k allocations: 638.781 KiB)


0.8262404466978552

In [64]:
ma = [[1 3 8 10]; [9 2 8 3]; [7 0 1 1]; [6 5 2 5]]
function I_n(
    n::Int64; 
    n_row::Int64 = 4
) 
    vec = Int64.(ones(n_row))
    vec[n] = 0
    return diagm(vec)
end

I_n(4) * ma * I_n(4)


4×4 Matrix{Int64}:
 1  3  8  0
 9  2  8  0
 7  0  1  0
 0  0  0  0

In [94]:
df_edges_testing = "/Users/jsyme/Documents/Projects/git/factset-interfirm-networks/factset_graph_optimization_model/ref/ensemble_model_new_edges_giovanni_20220929_complete/edges.csv"
df_edges_testing = read_csv(df_edges_testing, true);

field_vertex_i = :vertex_i
field_vertex_j = :vertex_j



v_i = df_edges_testing[:, field_vertex_i]
v_j = df_edges_testing[:, field_vertex_j]
n = maximum(union(v_i, v_j))

adj = sparse(v_i, v_j, ones(length(v_i))) 
adj = adj[1:5000, 1:5000] #TEMP: reduce for testing
graph = SimpleDiGraph(adj);


In [109]:
mat = [[0 1 0 1]; [1 0 0 0]; [0 0 0 1]; [1 0 1 0]]
g = SimpleGraph(mat)

{4, 3} undirected simple Int64 graph

In [16]:
?get_distance_matrices

search: get_distance_matrices



# Information

Return a distance mtarix and a lower-triangular distance matrix (without      diagonal). Returns a tuple of the form

```
`D`, `D_invs`

where `D` is the distance matrix (based on the shorted path) and `D_invs` is 
a matrix of distance inverses (excluding diagonal)
```

# Constructs

function get*distance*matrices(     adj::Union{SparseMatrixCSC, Matrix},     D::SharedArray{Float64};     digits_round::Int64 = 10 )::Tuple{Matrix{Float64}, Matrix{Float64}}

## Function Arguments

  * `adj`: sparse adjacency matrix used to set graph. Can be directed and/or    weighted

## Keyword Arguments

  * `digits_round`: number of digits to round distance matrix to. Used to chop    Infs to 0.
  * `graph`: optional graph associated with adj to pass


In [ ]:

"""
Swap indices S

- `S`: set of indices in current storage
- `S_in`: set of indices entering storage from adj
- `S_out`: set of indices exiting from storage back to 
    adj
- `arr_store_cols`: storage array storing the column vectors
    for subset of adj column indices S. Ordered by S.
- `arr_store_rows`: storage array storing the row vectors
    for subset of adj row indices S. Ordered by S.
- `arr_store_cols_transient`: intermediate storage array used 
    to facilitate column swaps
- `arr_store_rows_transient`: intermediate storage array used 
    to facilitate row swaps

"""
function swap!(
    adj::Union{SparseMatrixCSC{Float64, Int64}, Matrix{Float64}},
    S::Vector{Int64},
    S_in::Vector{Int64},
    S_out::Vector{Int64},
    arr_store_cols::Union{SparseMatrixCSC{Float64, Int64}, Matrix{Float64}},
    arr_store_rows::Union{SparseMatrixCSC{Float64, Int64}, Matrix{Float64}},
    arr_store_cols_transient::Matrix{Float64},
    arr_store_rows_transient::Matrix{Float64},
)
    if length(S_in) == length(S_out)
        
        n = length(S_in)
        inds_out = findall(x -> (x in S_out), S)
        
        arr_store_cols_transient = copy(arr_store_cols)
        arr_store_rows_transient = copy(arr_store_rows)
        
        
        ##  1. SWAP IN TO STORAGE FROM ADJACENCY
        
        inds_s_out = Int64.(zeros(n))
        
        for i in 1:n
            s_in = S_in[i]
            s_out = S_out[i]
            inds_s_out[i] = findall(x -> (x == s_out), S)[1]
            
            # column and column swaps
            if (s_out in S) & (s_in <= size(adj)[1])
                arr_store_cols[inds_s_out[i], :] = adj[:, s_in]
                arr_store_rows[inds_s_out[i], :] = adj[s_in, :]
            end
        end
        
        
        ##  2. SWAP OUT FROM TRANSIENT TO ADJACENCY & UPDATE S
        
        for i in 1:n
            s_in = S_in[i]
            s_out = S_out[i]

            if (s_out in S) & (s_in <= size(adj)[1])

                adj[:, s_out] = arr_store_cols_transient[inds_s_out[i], :]
                adj[s_out, :] = arr_store_rows_transient[inds_s_out[i], :]

                S[inds_s_out[i]] = s_in
            end
        end
    end
end


"""
# Constructs

stochastic_gradient_descent(
    n_nodes::Int64,
    graph_fast::test1_FastGraph;
    epsilon::Float64 = 10^(-6),
    max_iter::Int64 = 100,
    S::Union{Vector{Int64}, UnitRange{Int64}, Nothing} = nothing,
    randomize_swap_count::Bool = false
)

##  Function Arguments

- `n_nodes`: number of nodes to remove
- `graph_fast`: FastGraph storing information for running swaps


##  Keyword Arguments

- `distance_algorithm`: distance algorithm to use to update distance matrix. Passed to get_distance_matrices(). Default is :auto
- `epsilon`: convergence threshold
- `max_iter`: maximum number of iterations to allow
- `max_iter_no_improvement`: maximum number of iterations without improving the objective function to allow
- `print_info`: print outputs?
- `randomize_swap_count`: if true, randomizes number of swaps on each
    iteration. Otherwise, swaps one at a time
- `S`: optional set of indices to start with. Must have length `n_nodes`

"""
function stochastic_gradient_descent_depr(
    n_nodes::Int64,
    graph_fast::test1_FastGraph;
    distance_algorithm::Symbol = :auto,
    epsilon::Float64 = 10^(-6),
    max_iter::Int64 = 1000,
    max_iter_no_improvement::Int64 = 200,
    print_info::Bool = false,
    S::Union{Vector{Int64}, UnitRange{Int64}, Nothing} = nothing,
    randomize_swap_count::Bool = false
)
    # setup 
    adj = copy(graph_fast.A)
    m, n = size(adj)
    symmetric_q = LinearAlgebra.issymmetric(adj)
    max_iter_no_improvement = minimum([max_iter_no_improvement, max_iter])
    
    # conditions to exit on
    return_nothing = (m != n)
    return_nothing |= (m < n_nodes)
    return_nothing |= (max_iter_no_improvement < 0)
    return_nothing |= (max_iter < 0)
    
    if return_nothing#(m != n) | (m < n_nodes) | (max_iter_no_improvement < 0) | (max_iter < 0)
        return nothing
    end
    
    
    all_vertices = collect(1:m)
    
    # initialize set of states
    S = isa(S, Nothing) ? sample(all_vertices, n_nodes; replace = false) : S
    S = issubset(Set(S), Set(all_vertices)) ? S : sample(all_vertices, n_nodes; replace = false)
    sort!(S)
    
    # set up storage for swaps and initialize from random S - need the storage arrays and the arrays to use to facilitate swaps (transient)
    arr_store_cols = zeros(n_nodes, m)
    arr_store_rows = zeros(n_nodes, m)
    arr_store_cols_transient = zeros(n_nodes, m)
    arr_store_rows_transient = zeros(n_nodes, m)
    
    function swap_sgd!(
        S_sgd::Vector{Int64},
    )
        swap!(
            adj,
            S_sgd, 
            S_sgd, 
            S_sgd,
            arr_store_cols,
            arr_store_rows,
            arr_store_cols_transient,
            arr_store_rows_transient,
        )
    end
    
    # initialize by swapping all rows/cols into storage
    swap_sgd!(S)

    
    # get current fragmentation
    F = fragmentation(
        graph_fast.A;
        D_invs = graph_fast.D_inv,
        graph = graph_fast.graph
    )
        
    F_0 = 0
    i = 0 # number of iterations
    i_no_improvement = 0 # number of iterations with no improvement in objective function
    s = 0
    F_try = 0
    
    
    while (i < max_iter) & (F - F_try >= epsilon) & (i_no_improvement < max_iter_no_improvement)#(F - F_0 >= epsilon)
        
        swap_size = randomize_swap_count ? sample(1:n_nodes, 1; replace = false)[1] : 1
        s_out = Vector{Int64}(sample(S, swap_size; replace = false))
        s_in = Vector{Int64}(sample(setdiff(all_vertices, S), swap_size; replace = false))
        S_try = Vector{Int64}(sort(union(setdiff(S, s_out), s_in)))
        
        # swap everything back, then swap in new set and check fragmentation
        swap_sgd!(S) 
        swap_sgd!(S_try)
        
        # recalculate distances
        graph = symmetric_q ? SimpleGraph(adj) : SimpleDiGraph(adj)
        d, d_inv = get_distance_matrices(
            adj; 
            algorithm = distance_algorithm,
            graph = graph
        )
        
        F_try = fragmentation(
            adj;
            D_invs = d_inv,
            graph = graph
        )
        
            
        if F_try <= F
            # swap back if no good
            swap_sgd!(S_try)
            swap_sgd!(S)
        else
            print_info ? print("\n\n** keeping swap! $(i) - F = $(F), F_0 = $(F_0), F_try = $(F_try)\n") : nothing
            F_0 = F
            F = F_try
            S = S_try
            i_no_improvement = -1
        end
        
        i += 1
        i_no_improvement += 1
        
        ((i%10 == 0) & print_info) ? println("$(i) iterations complete with value $(F) (previous $(F_0))") : nothing
    end
    
    ((i < max_iter) & print_info) ? println("Converged in $(i) iterations.") : nothing
    
    #return nothing
    return adj, arr_store_cols, arr_store_rows, S, F
end

In [65]:
subgraph = connected_components(graph)[1]
adj_red = adj[subgraph, subgraph];
subgraph = SimpleDiGraph(adj_red);



In [18]:
#
# START WITH WHATEVER'S QUICKEST
#   LOOK AT PYNTACLE 
#
# synthetic annealing, tabboo, stochastic gradient descent, linear programming, cross entropy, genetic algorithms

In [19]:
#

In [ ]:
#